In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [3]:
df = pd.read_csv("/content/drive/MyDrive/house_price_regression_dataset.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Square_Footage        1000 non-null   int64  
 1   Num_Bedrooms          1000 non-null   int64  
 2   Num_Bathrooms         1000 non-null   int64  
 3   Year_Built            1000 non-null   int64  
 4   Lot_Size              1000 non-null   float64
 5   Garage_Size           1000 non-null   int64  
 6   Neighborhood_Quality  1000 non-null   int64  
 7   House_Price           1000 non-null   float64
dtypes: float64(2), int64(6)
memory usage: 62.6 KB


In [5]:
X = df.drop("House_Price",axis = 1).values
y = df["House_Price"].values

In [ ]:
X.shape

(1000, 7)

In [ ]:
y.shape

(1000,)

In [ ]:
X

array([[1.36000000e+03, 2.00000000e+00, 1.00000000e+00, ...,
        5.99636640e-01, 0.00000000e+00, 5.00000000e+00],
       [4.27200000e+03, 3.00000000e+00, 3.00000000e+00, ...,
        4.75301385e+00, 1.00000000e+00, 6.00000000e+00],
       [3.59200000e+03, 1.00000000e+00, 2.00000000e+00, ...,
        3.63482272e+00, 0.00000000e+00, 9.00000000e+00],
       ...,
       [2.60600000e+03, 4.00000000e+00, 2.00000000e+00, ...,
        4.05506679e+00, 0.00000000e+00, 2.00000000e+00],
       [4.72300000e+03, 5.00000000e+00, 2.00000000e+00, ...,
        1.93092148e+00, 0.00000000e+00, 7.00000000e+00],
       [3.26800000e+03, 4.00000000e+00, 2.00000000e+00, ...,
        3.10879025e+00, 2.00000000e+00, 2.00000000e+00]])

# normalizing the dataset




In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split before scaling (to avoid data leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)


# Scale features
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Reshape target to 2D (n_samples, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Scale targets
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [ ]:
X

array([[1.36000000e+03, 2.00000000e+00, 1.00000000e+00, ...,
        5.99636640e-01, 0.00000000e+00, 5.00000000e+00],
       [4.27200000e+03, 3.00000000e+00, 3.00000000e+00, ...,
        4.75301385e+00, 1.00000000e+00, 6.00000000e+00],
       [3.59200000e+03, 1.00000000e+00, 2.00000000e+00, ...,
        3.63482272e+00, 0.00000000e+00, 9.00000000e+00],
       ...,
       [2.60600000e+03, 4.00000000e+00, 2.00000000e+00, ...,
        4.05506679e+00, 0.00000000e+00, 2.00000000e+00],
       [4.72300000e+03, 5.00000000e+00, 2.00000000e+00, ...,
        1.93092148e+00, 0.00000000e+00, 7.00000000e+00],
       [3.26800000e+03, 4.00000000e+00, 2.00000000e+00, ...,
        3.10879025e+00, 2.00000000e+00, 2.00000000e+00]])

# Initializing weights


In [31]:
input_size = X_train.shape[1]
hidden1_size = 64
hidden2_size = 32
output_size = 1

In [19]:
import random

W1 = np.random.randn(input_size,hidden1_size)*np.sqrt(2/input_size)
b1 = np.zeros((1,hidden1_size))

W2 = np.random.randn(hidden1_size,hidden2_size)*np.sqrt(2/hidden1_size)
b2 = np.zeros((1,hidden2_size))

W3 = np.random.randn(hidden2_size,output_size)*np.sqrt(2/hidden2_size)
b3 = np.zeros((1,output_size))


In [ ]:
W3.shape

(32, 1)

# Helping function

In [20]:
def relu(Z):
    return np.maximum(0,Z)


def relu_derivative(Z):
    return (Z>0).astype(float)

def mse(y_true,y_pred):
    return np.mean((y_true - y_pred)** 2)

def mse_derivative(y_true,y_pred):
    return (2 * (y_pred - y_true)) / y_true.size

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# forward Propagation

In [21]:
def forward(X):
    Z1 = X @ W1 + b1
    A1 = relu(Z1)

    Z2 = A1 @ W2 + b2
    A2 = relu(Z2)

    Z3 = A2 @ W3 + b3

    cache = (Z1,A1,Z2,A2,Z3)

    return Z3,cache

# Backward Propagation

In [11]:
def backward(X,y,cache):
    Z1,A1,Z2,A2,Z3 = cache
    m = y.shape[0]

    # output layer

    dZ3 = mse_derivative(y,Z3)
    dW3 = A2.T @ dZ3 / m
    db3 = np.sum(dZ3,axis = 0,keepdims= True)/m

    # hidden layer2

    dA2 = dZ3 @ W3.T
    dZ2 = dA2 * relu_derivative(Z2)
    dW2 = A1.T @ dZ2 / m
    db2 = np.sum(dZ2,axis = 0,keepdims = True) / m

    # hidden layer1

    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = X.T @ dZ1 / m
    db1 = np.sum(dZ1,axis = 0,keepdims = True) / m



    return dW1, db1, dW2, db2, dW3, db3

# Training loop

In [32]:
epochs = 500
lr = 0.01  # try larger LR

for epoch in range(epochs):
    # Forward pass
    y_pred_scaled, cache = forward(X_train)

    # Compute loss on scaled values
    loss = mse(y_train_scaled, y_pred_scaled)

    # Backward pass
    dW1, db1, dW2, db2, dW3, db3 = backward(X_train, y_train_scaled, cache)

    # Update weights
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    W3 -= lr * dW3
    b3 -= lr * db3

    # For reporting, compute MAPE in original scale
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    mape = mean_absolute_percentage_error(y_train, y_pred)

    print(f"Epoch {epoch+1}/{epochs}, MSE: {loss:.6f}, MAPE: {mape:.2f}%")


Epoch 1/500, MSE: 1.260611, MAPE: 44.40%
Epoch 2/500, MSE: 1.260511, MAPE: 44.40%
Epoch 3/500, MSE: 1.260411, MAPE: 44.40%
Epoch 4/500, MSE: 1.260311, MAPE: 44.40%
Epoch 5/500, MSE: 1.260211, MAPE: 44.40%
Epoch 6/500, MSE: 1.260111, MAPE: 44.40%
Epoch 7/500, MSE: 1.260012, MAPE: 44.40%
Epoch 8/500, MSE: 1.259912, MAPE: 44.40%
Epoch 9/500, MSE: 1.259812, MAPE: 44.40%
Epoch 10/500, MSE: 1.259713, MAPE: 44.40%
Epoch 11/500, MSE: 1.259613, MAPE: 44.40%
Epoch 12/500, MSE: 1.259514, MAPE: 44.40%
Epoch 13/500, MSE: 1.259414, MAPE: 44.40%
Epoch 14/500, MSE: 1.259315, MAPE: 44.40%
Epoch 15/500, MSE: 1.259215, MAPE: 44.40%
Epoch 16/500, MSE: 1.259116, MAPE: 44.40%
Epoch 17/500, MSE: 1.259016, MAPE: 44.40%
Epoch 18/500, MSE: 1.258917, MAPE: 44.40%
Epoch 19/500, MSE: 1.258818, MAPE: 44.40%
Epoch 20/500, MSE: 1.258718, MAPE: 44.40%
Epoch 21/500, MSE: 1.258619, MAPE: 44.40%
Epoch 22/500, MSE: 1.258520, MAPE: 44.40%
Epoch 23/500, MSE: 1.258421, MAPE: 44.40%
Epoch 24/500, MSE: 1.258322, MAPE: 44.40%
E

In [33]:
y_test_pred_scaled, _ = forward(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

# MSE and MAPE on test set
test_mse = mse(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"Test MSE: {test_mse:.2f}")
print(f"Test MAPE: {test_mape:.2f}%")


Test MSE: 82405678305.63
Test MAPE: 47.13%
